# Libraries

In [1]:
import flirimageextractor
from matplotlib import cm
import os
import numpy as np
import warnings
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score
warnings.filterwarnings("ignore", category=RuntimeWarning)

2023-05-02 13:47:17.892856: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 13:47:21.899085: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-02 13:47:27.665644: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-02 13:47:27.665925: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# Functions

In [2]:
class preprocess():
    def __init__(self):
        self.model = flirimageextractor.FlirImageExtractor(palettes=[cm.jet, cm.bwr, cm.gist_ncar])
        
    def extract_thermal_image(self, img_path):
        self.model.process_image(img_path)
        return self.model.get_thermal_np()

# Preprocessiong

In [3]:
base_path = 'data/'
prep = preprocess()
data = {}
for classes in os.listdir(base_path):
    img_path = base_path+classes+'/'
    cls = int(classes)
    if  data.get(cls) is None:
        data[cls] = []

    for images in os.listdir(img_path):
        fmt = images.split('.')

        if fmt[-1] == "jpg":
            id = fmt[0]
            image_path = img_path + images
            data[cls].append(prep.extract_thermal_image(image_path).copy())

# Data Preparation

In [4]:
n_train = 4
Y_train = np.empty((0), int)
Y_test = np.empty((0), int)
X_train = np.empty((0,140,140), float)
X_test = np.empty((0,140,140), float)

for label in data:
    if label != 90:
        n = len(data[label])
        Y_train = np.append(Y_train, [label]*n_train, axis=0)
        X_train = np.append(X_train, data[label][0:n_train], axis=0)
        n -= n_train
        Y_test = np.append(Y_test, [label]*n, axis=0)
        X_test = np.append(X_test, data[label][n_train:], axis=0)
    else:
        dtr = np.array(data[label][0])
        dtr = dtr[np.newaxis,:]
        dte = np.array(data[label][1])
        dte = dte[np.newaxis,:]
        Y_train = np.append(Y_train, [label], axis=0)
        X_train = np.append(X_train, dtr, axis=0)
        Y_test = np.append(Y_test, [label], axis=0)
        X_test = np.append(X_test, dte, axis=0)

X_train = X_train/100
X_test = X_test/100

Y = np.zeros((11, len(Y_train)))
for i in range(0,len(Y_train)):
    Y[Y_train[i]//10,i] = 1

In [5]:
def create_cnn(width=144, height=144, depth=1, filters=(4, 8, 16)):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    # x = tf.keras.layers.Dense(128)(x)
    # x = tf.keras.layers.Activation("relu")(x)
    # x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    # x = tf.keras.layers.Dropout(0.5)(x)
    # x = tf.keras.layers.Dense(64)(x)
    # x = tf.keras.layers.Activation("relu")(x)
    # x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    # x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(16)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(11)(x)
    x = tf.keras.layers.Activation("softmax")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model 

In [6]:
model = create_cnn(140, 140, 1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=X_train, y=Y.T, epochs=200, batch_size=5, verbose=0)

2023-05-02 13:48:56.534667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 13:48:59.296889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 13:48:59.296982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-02 13:48:59.304971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [7]:
preds = model.predict(X_test)
y_pred = np.zeros(len(preds))

for i in range(0,len(preds)):
    y_pred[i] = 10 * np.argmax(preds[i,:])

# print("True values: ", Y_test)
# print("Predicted values:", y_pred)
print("Confusion Matrix :")
print(confusion_matrix(Y_test, y_pred))
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(Y_test, y_pred)))

4/4 [==============================] - 1s 65ms/step
Confusion Matrix :
[[ 0  0  7  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  0  0]]

Accuracy: 0.09



: 